# Police Radio Analysis: Data Collection (using Local Library)
*** 

### Contents:
- [Overview](#Overview)
- [Dataframe by Word](#Data-Collection:-Single-Word-Observation)
- [Dataframe by Sentence](#Data-Collection:-Sentence-by-Speaker-Observation)
- [Dataframe by Location (for Mapping)](#Data-Collection:-Threatened-Locations)

## Overview

- Audio Files: Broadcastify archives for 11/08/2018 between 6:44AM-9:12AM in Butte County, CA (The Camp Fire)


- Used Amazon Transcribe to transcribe audio to text (output JSON file)


- Steps to convert JSON files to a structured dataframes:
    1. Take word and speaker information from individual dataframe to create two dataframes
    2. Merge the two dataframes by assigning the speaker to the word based on the time
    3. Based on speaker and start/end times reconstruct sentences (new speaker signifies new observation)
    4. Repeat for every JSON file and combine all into on dataframe
    5. Add additional desired columns based on current information in dataframe:
        - Clean version of text (no punctuation, all lowercase)
        - Start and stop time as datetime objects using the initial start time in the file name
        - Indicator for containing 'fire' or 'evacuation' in the observations
    6. Sort dataframe by the start time (datetime object) and export
    7. Use existing dataframe and to make a new dataframe where each identified location is one observation


- Output: 3 Dataframes
    - Dataframe by Individual Words
    - Dataframe by Sentences
    - Dataframe by Locations Mentioned (for Mapping)

#### Import Libraries:

In [1]:
import pandas as pd
import glob
import re
import geopandas as gpd

# Local Library
import data_collection

In [2]:
data_collection.get_dataframe

<function data_collection.get_dataframe(file_name, ca_places)>

#### Load Geographic Data for Mapping

In [3]:
## Loads geographic data from https://data.ca.gov/dataset/ca-geographic-boundaries
ca_places = gpd.read_file('./data/CA_places/CA_Places_TIGER2016.shp')
ca_state = gpd.read_file('./data/CA_State/CA_State_TIGER2016.shp')
ca_counties = gpd.read_file('./data/CA_Counties/CA_Counties_TIGER2016.shp')

## Reset INTPTLAT & INTPTLON to floats
ca_places['INTPTLAT'] = ca_places['INTPTLAT'].astype('float')
ca_places['INTPTLON'] = ca_places['INTPTLON'].astype('float')
ca_state['INTPTLAT'] = ca_state['INTPTLAT'].astype('float')
ca_state['INTPTLON'] = ca_state['INTPTLON'].astype('float')
ca_counties['INTPTLAT'] = ca_counties['INTPTLAT'].astype('float')
ca_counties['INTPTLON'] = ca_counties['INTPTLON'].astype('float')

## Reset places dataframe to capital NAME
for i, row in enumerate(ca_places.iterrows()):
    ca_places.loc[i,'NAME'] = row[1]['NAME'].upper()

## Data Collection: Single Word Observation

In [4]:
# Get word dataframe

# Use glob to get list of all json files in the folder
files_json = (glob.glob('./data/translations/*.json'))

# Create empty dataframe
words = pd.DataFrame()
rows = 0 

for i, file_name in enumerate(files_json):
    word_one_file, _ = data_collection.transcription_outputs(file_name)
    
    rows += len(word_one_file)
    
    # Print status
    print(f'{i+1} of {len(files_json)} Dataframes Completed ({rows} rows): {file_name}')
    
    # Add each dataframe together
    words = pd.concat([words, word_one_file])
    
# Reset index of master dataframe
words.reset_index(drop=True, inplace=True)

# Print shape of dataframe after going through all JSON files
print(f'Words Dataframe Shape: {words.shape}')

1 of 20 Dataframes Completed (558 rows): ./data/translations/201811081001-444704-22956_.json
2 of 20 Dataframes Completed (1146 rows): ./data/translations/201811080929-467022-25641_.json
3 of 20 Dataframes Completed (1201 rows): ./data/translations/201811080858-659667-24574_.json
4 of 20 Dataframes Completed (2511 rows): ./data/translations/201811080931-763045-22956_.json
5 of 20 Dataframes Completed (3921 rows): ./data/translations/201811080901-584135-22956_.json
6 of 20 Dataframes Completed (4113 rows): ./data/translations/201811081011-319947-26936_.json
7 of 20 Dataframes Completed (4253 rows): ./data/translations/201811080911-136992-26936_.json
8 of 20 Dataframes Completed (5080 rows): ./data/translations/201811081012-237044-1929_.json
9 of 20 Dataframes Completed (5143 rows): ./data/translations/201811080959-402082-25641_.json
10 of 20 Dataframes Completed (5208 rows): ./data/translations/201811080928-650127-24574_.json
11 of 20 Dataframes Completed (5637 rows): ./data/translation

In [5]:
words.head()

,content,confidence,start_time,end_time,type,feed
0,wait,0.3086,38.04,39.28,pronunciation,201811081001-444704-22956_
1,FIRE,0.9876,39.29,39.88,pronunciation,201811081001-444704-22956_
2,with,1.0,39.88,40.64,pronunciation,201811081001-444704-22956_
3,every,0.1804,41.22,41.41,pronunciation,201811081001-444704-22956_
4,night,0.2812,41.41,41.72,pronunciation,201811081001-444704-22956_


In [6]:
# Save words dataframe

# Export: save as csv
words.to_csv('./data/words.csv', index=False)

# Export: save as pkl
words.to_pickle('./data/words.pkl')

## Data Collection: Sentence by Speaker Observation

In [7]:
# Get sentence dataframe

# Use glob to get list of all json files in the folder
files_json = (glob.glob('./data/translations/*.json'))

# Create empty dataframe
sentence = pd.DataFrame()
rows = 0 

# Iterate through the files and get a dataframe for each file
for i, file_name in enumerate(files_json):
    
    # Get dataframe for an individual file
    df_one_file = data_collection.get_dataframe(file_name, ca_places)
    rows += len(df_one_file)
    
    # Print status
    print(f'{i+1} of {len(files_json)} Dataframes Completed (new rows: {len(df_one_file)}, total rows: {rows}): {file_name}')
    
    # Add each dataframe together
    sentence = pd.concat([sentence, df_one_file])

# Reset index of master dataframe
sentence.reset_index(drop=True, inplace=True)

# Print shape of dataframe after going through all JSON files
print(f'Sentence Dataframe Shape: {sentence.shape}')

1 of 20 Dataframes Completed (new rows: 52, total rows: 52): ./data/translations/201811081001-444704-22956_.json
2 of 20 Dataframes Completed (new rows: 44, total rows: 96): ./data/translations/201811080929-467022-25641_.json
3 of 20 Dataframes Completed (new rows: 6, total rows: 102): ./data/translations/201811080858-659667-24574_.json
4 of 20 Dataframes Completed (new rows: 50, total rows: 152): ./data/translations/201811080931-763045-22956_.json
5 of 20 Dataframes Completed (new rows: 30, total rows: 182): ./data/translations/201811080901-584135-22956_.json
6 of 20 Dataframes Completed (new rows: 10, total rows: 192): ./data/translations/201811081011-319947-26936_.json
7 of 20 Dataframes Completed (new rows: 7, total rows: 199): ./data/translations/201811080911-136992-26936_.json
8 of 20 Dataframes Completed (new rows: 45, total rows: 244): ./data/translations/201811081012-237044-1929_.json
9 of 20 Dataframes Completed (new rows: 2, total rows: 246): ./data/translations/201811080959

#### Add Additional Columns

In [8]:
# Add column for clean text (remove punctuation and make all lowercase)
# Reference: Code adapted from NLP_EDA-InClass in DEN Flex by Sam Stack
def clean_text(raw_text):
    words = re.sub(r'[^a-z0-9]', r' ', raw_text.lower()).split()
    return ' '.join(words)

sentence['text_clean'] = sentence['text'].apply(clean_text)

In [11]:
# Add columns for start and end time as datetime objects (using speaker_start/end and file name)
start_time = []
end_time = []

for i in range(len(sentence)):
    start_time.append(data_collection.actual_time_str(sentence['speaker_start'][i], sentence['feed'][i]))
    end_time.append(data_collection.actual_time_str(sentence['speaker_end'][i], sentence['feed'][i]))

sentence['start_time'] = start_time
sentence['end_time'] = end_time

In [12]:
# Add columns that indicate if fire or evacuation related words were mentioned in that observation
sentence['contains_fire'] = sentence['text_clean'].map(lambda x: 1 if 'fire' in x else 0)
sentence['contains_evac'] = sentence['text_clean'].map(lambda x: 1 if 'evac' in x else 0)

In [13]:
# Look at dataframe
sentence.head(3)

,text,speaker_start,speaker_end,speaker_length,speaker,sentence,word_confidence,avg_confidence,min_conf,feed,...,feed_name,department,INTPTLON,INTPTLAT,ID_PLACES,text_clean,contains_fire,contains_evac,start_time,end_time
0,wait FIRE with every night .,52.94,53.39,0.45,spk_0,4,"[0.3086, 0.9876, 1.0, 0.1804, 0.2812, nan]",0.55156,0.1804,201811081001-444704-22956_,...,Chico_Paradise_Fire__CalFire,FIRE,[],[],[],wait fire with every night,1,0,2018-11-08 08:01:52-08:00,2018-11-08 08:01:53-08:00
1,Thirty,53.39,53.7,0.31,spk_2,5,[0.6417],0.64170,0.6417,201811081001-444704-22956_,...,Chico_Paradise_Fire__CalFire,FIRE,[],[],[],thirty,0,0,2018-11-08 08:01:53-08:00,2018-11-08 08:01:53-08:00
2,nine quarters . Affirmative evacuation order i...,78.64,78.85,0.21,spk_0,12,"[0.9994, 0.5192, nan, 0.8876, 0.9996, 0.8254, ...",0.82390,0.2566,201811081001-444704-22956_,...,Chico_Paradise_Fire__CalFire,FIRE,[],[],[],nine quarters affirmative evacuation order is ...,0,1,2018-11-08 08:02:18-08:00,2018-11-08 08:02:18-08:00


In [14]:
# Verify results of start and end time to datatime objects
print(sentence[['start_time']].min())
print(sentence[['start_time']].max())
print(sentence[['end_time']].max())

start_time   2018-11-08 06:44:31-08:00
dtype: datetime64[ns, US/Pacific]
start_time   2018-11-08 09:12:25-08:00
dtype: datetime64[ns, US/Pacific]
end_time   2018-11-08 09:12:26-08:00
dtype: datetime64[ns, US/Pacific]


In [15]:
# Sort dataframe by start time
sentence.sort_values(by = 'start_time', inplace = True)

# Reset index
sentence.reset_index(drop=True, inplace=True)

In [16]:
# Verify changes
print('Sentence Shape: ', sentence.shape)
sentence.index

Sentence Shape:  (464, 22)


RangeIndex(start=0, stop=464, step=1)

In [17]:
# Save words dataframe

# Export: save as csv
sentence.to_csv('./data/sentence.csv', index=False)

# Export: save as pkl
sentence.to_pickle('./data/sentence.pkl')

## Data Collection: Threatened Locations

In [18]:
# Creates a dataframe from the sentence dataframe that only includes the locations mentioned/threatened
threat = data_collection.create_threat_df(sentence)

In [19]:
threat.head()

,latitude,longitude,id_places,text,confidence,feed,start_time,end_time,department
0,39.7542,-121.606,PARADISE,justin maguire is clear and counting down from...,0.806101,Oroville_Police_Fire,2018-11-08 06:59:15-08:00,2018-11-08 06:59:17-08:00,BOTH
1,37.4829,-118.602,PARADISE,justin maguire is clear and counting down from...,0.806101,Oroville_Police_Fire,2018-11-08 06:59:15-08:00,2018-11-08 06:59:17-08:00,BOTH
2,39.4955,-121.56,OROVILLE,left thirty eleven the one you re with and thi...,0.784975,Chico_Paradise_Fire__CalFire,2018-11-08 07:04:34-08:00,2018-11-08 07:04:34-08:00,FIRE
3,39.6315,-121.405,BERRY CREEK,left thirty eleven the one you re with and thi...,0.784975,Chico_Paradise_Fire__CalFire,2018-11-08 07:04:34-08:00,2018-11-08 07:04:34-08:00,FIRE
4,39.7703,-121.513,CONCOW,left thirty eleven the one you re with and thi...,0.784975,Chico_Paradise_Fire__CalFire,2018-11-08 07:04:34-08:00,2018-11-08 07:04:34-08:00,FIRE


In [20]:
# Export: save as csv
threat.to_csv('./data/threat.csv', index=False)

# Export: save as pkl
threat.to_pickle('./data/threat.pkl')